In [0]:
!pip install pyrouge
!pip install pytorch_transformers 

     |████████████████████████████████| 184kB 6.5MB/s 
     |████████████████████████████████| 1.0MB 38.5MB/s 
     |████████████████████████████████| 860kB 36.8MB/s 
     |████████████████████████████████| 655kB 40.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=58ce999505f4147ba1ce2e6c058f3783ae36d41adee4b77209e45003feacd164
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609239 sha256=51d1a41128ac4a8e475475ef7e67ab6b69a76a0bc908388e644b653104f37c48
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built sacremoses regex


In [0]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Writing setup.sh


In [0]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 5464 (delta 9), reused 19 (delta 2), pack-reused 5422
Receiving objects: 100% (5464/5464), 13.34 MiB | 12.70 MiB/s, done.
Resolving deltas: 100% (3520/3520), done.
Created temporary directory: /tmp/pip-ephem-wheel-cache-dzcfq_ab
Created temporary directory: /tmp/pip-req-tracker-s12rbzax
Created requirements tracker '/tmp/pip-req-tracker-s12rbzax'
Created temporary directory: /tmp/pip-install-7akk11cw
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-5qx9z7ky
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-s12rbzax'
    Running setup.py (path:/tmp/pip-req-build-5qx9z7ky/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.2.0
    running egg_info
    creating pip-egg-info/apex.egg-info
    writing p

In [0]:
! rm -rf *
! git clone https://github.com/microsoft/unilm.git
! mv unilm/src/* ./
! rm -rf unilm

Cloning into 'unilm'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 78 (delta 31), reused 31 (delta 6), pack-reused 0
Unpacking objects: 100% (78/78), done.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
! mkdir ./DATA/
! unzip  './drive/My Drive/Colab Notebooks/cnndm_data.zip' -d ./DATA/
! mkdir ./MODELS/
! unzip './drive/My Drive/Colab Notebooks/cnndm_model.zip' -d ./MODELS/
! mkdir ./TMP/

mkdir: cannot create directory ‘./DATA/’: File exists
Archive:  ./drive/My Drive/Colab Notebooks/cnndm_data.zip
  inflating: ./DATA/dev.src          
  inflating: ./DATA/dev.tgt          
   creating: ./DATA/org_data/
  inflating: ./DATA/org_data/dev.article  
  inflating: ./DATA/org_data/dev.summary  
  inflating: ./DATA/org_data/test.article  
  inflating: ./DATA/org_data/test.summary  
  inflating: ./DATA/org_data/training.article  
  inflating: ./DATA/org_data/training.summary  
  inflating: ./DATA/test.src         
  inflating: ./DATA/test.tgt         
  inflating: ./DATA/train.src        
  inflating: ./DATA/train.tgt        
mkdir: cannot create directory ‘./MODELS/’: File exists
Archive:  ./drive/My Drive/Colab Notebooks/cnndm_model.zip
  inflating: ./MODELS/cnndm_model.bin  




```
DATA_DIR=/{path_of_data}/cnn_dailymail
MODEL_RECOVER_PATH=/{path_of_fine-tuned_model}/cnndm_model.bin
EVAL_SPLIT=test
export PYTORCH_PRETRAINED_BERT_CACHE=/{tmp_folder}/bert-cased-pretrained-cache
# run decoding
python biunilm/decode_seq2seq.py --fp16 --amp --bert_model bert-large-cased --new_segment_ids --mode s2s --need_score_traces \
  --input_file ${DATA_DIR}/${EVAL_SPLIT}.src --split ${EVAL_SPLIT} --tokenized_input \
  --model_recover_path ${MODEL_RECOVER_PATH} \
  --max_seq_length 768 --max_tgt_length 128 \
  --batch_size 64 --beam_size 5 --length_penalty 0 \
  --forbid_duplicate_ngrams --forbid_ignore_word ".|[X_SEP]"
# apply length penalty
python biunilm/gen_seq_from_trace.py --bert_model bert-large-cased --alpha 1.0 \
  --input ${MODEL_RECOVER_PATH}.${EVAL_SPLIT}
# run evaluation
python gigaword/eval.py --pred ${MODEL_RECOVER_PATH}.${EVAL_SPLIT}.alp1.0 \
  --gold ${DATA_DIR}/org_data/${EVAL_SPLIT}.summary --trunc_len 70 --perl
```



In [0]:
DATA_DIR='./DATA/'
MODEL_RECOVER_PATH='./MODELS/cnndm_model.bin'
EVAL_SPLIT='test'
tmp_folder = "TMP"
! export PYTORCH_PRETRAINED_BERT_CACHE=/{tmp_folder}/bert-cased-pretrained-cache
! touch __init__.py

In [0]:
from pytorch_pretrained_bert.tokenization import BertTokenizer, WhitespaceTokenizer

In [0]:
# run decoding
! python biunilm/decode_seq2seq.py --fp16 --amp --bert_model bert-large-cased --new_segment_ids --mode s2s --need_score_traces \
  --input_file ${DATA_DIR}/${EVAL_SPLIT}.src --split ${EVAL_SPLIT} --tokenized_input \
  --model_recover_path ${MODEL_RECOVER_PATH} \
  --max_seq_length 768 --max_tgt_length 128 \
  --batch_size 64 --beam_size 5 --length_penalty 0 \
  --forbid_duplicate_ngrams --forbid_ignore_word ".|[X_SEP]"


Traceback (most recent call last):
  File "biunilm/decode_seq2seq.py", line 20, in <module>
    from pytorch_pretrained_bert.tokenization import BertTokenizer, WhitespaceTokenizer
ModuleNotFoundError: No module named 'pytorch_pretrained_bert'


In [0]:
# apply length penalty
! python biunilm/gen_seq_from_trace.py --bert_model bert-large-cased --alpha 1.0 \
  --input ${MODEL_RECOVER_PATH}.${EVAL_SPLIT}


Traceback (most recent call last):
  File "biunilm/gen_seq_from_trace.py", line 9, in <module>
    from pytorch_pretrained_bert.tokenization import BertTokenizer
ModuleNotFoundError: No module named 'pytorch_pretrained_bert'


In [0]:
# run evaluation
!python gigaword/eval.py --pred ${MODEL_RECOVER_PATH}.${EVAL_SPLIT}.alp1.0 \
  --gold ${DATA_DIR}/org_data/${EVAL_SPLIT}.summary --trunc_len 70 --perl

Traceback (most recent call last):
  File "gigaword/eval.py", line 19, in <module>
    import rouge
ModuleNotFoundError: No module named 'rouge'
